# LangChain

(deels geïnspireerd door een lessenreeks van DeepLearning.ai)

Voor we starten gaan we eerst zorgen dat iedereen een werkende setup heeft. Als je vorige keer OpenAI opgezet hebt, en je api key staat nog altijd mooi in de `.env` dan is alles oké.
Als je credits op waren, is er een alternatief via [Eden AI](https://www.edenai.co/), Eden AI heeft als business model om via één centrale API (de hunne) allerlei andere AI API's aan te spreken. Maar wat voor ons vooral handig is, is dat je een account kan aanmaken zonder credit card, en dan $10 gratis credits krijgt (en dus kunnen we via hen OpenAI aanspreken)

Ga naar [www.edenai.co](https://www.edenai.co/) en maak een account aan.

<div>
    <img src="img/edenai_signup.png" style="width: 700px;" >
</div>


Dan kan je bij Account Management je API key vinden

<div>
    <img src="img/edenai_apikey.png" style="width: 700px;" >
</div>



Bewaar nu deze API key ook in je `.env` file (de .env file moet ergens in een parent folder van al je notebooks staan, dus als je de repo gewoon gecloned hebt gewoon in de `2324-trendsinai/` folder zelf

```bash
OPENAI_API_KEY=sk-YYYYY
EDENAI_API_KEY=XXXXXXXX
```

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key = os.getenv('OPENAI_API_KEY')
edenai_api_key = os.getenv('EDENAI_API_KEY')

# print (openai.api_key)
# print (edenai_api_key)


Als je de print commando's uitvoert, zou je de key(s) die je wilt gebruiken moeten zien, als je `None` ziet staan is er iets mis.

De key van EdenAI is een Bearer token, daar kennen jullie alles van uit de Webservices cursus. Rechtstreeks de EdenAI API aanspreken doe je met een POST request naar de juiste url.
EdenAI heeft véél endpoints, een ChatGPT request zou er als volgt uitzien:

In [2]:
import requests

url = "https://api.edenai.run/v2/text/chat"

payload = {
    "response_as_dict": True,
    "attributes_as_list": False,
    "show_original_response": False,
    "temperature": 0,
    "max_tokens": 1000,
    "providers": "openai",
    "text": "what are the advantages of langchain?"
}
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": f"Bearer {edenai_api_key}"
}

response = requests.post(url, json=payload, headers=headers)

print(response.text)

{"openai":{"status":"success","generated_text":"LangChain offers several advantages:\n\n1. Accessibility: LangChain provides a decentralized platform that connects language learners and tutors from around the world. This accessibility allows learners to find tutors who are native speakers of the language they want to learn, ensuring a more authentic learning experience.\n\n2. Cost-effective: By eliminating intermediaries and connecting learners directly with tutors, LangChain reduces the cost of language learning. Learners can find tutors at affordable rates, making language learning more accessible to a wider audience.\n\n3. Flexibility: LangChain offers flexibility in terms of scheduling and learning pace. Learners can choose tutors based on their availability and can schedule lessons at their convenience. This flexibility allows learners to fit language learning into their busy schedules.\n\n4. Personalized learning: LangChain enables personalized learning experiences by connecting 

We krijgen een string waar ogenschijnlijk wel een JSON-object in zit met key `openai` (we kunnen dit ook direct naar meerdere LLM's sturen en dan zien we elke response), een `status` (hopelijk success) en dan de `generated_text`, er volgt ook nog een array met de role-user, role-assistant historiek in het geval van openai, en op het einde een `cost`.

Toen ik deze vraag stelde, kostte het antwoord mij 0.000708; dus met onze 10 dollar kunnen we een 15.000 van dat soort requests sturen, veel, maar ook niet oneindig.

We moeten dit JSON-object wel eerst omzetten naar Python, voor we verder kunnen.

In [3]:
import json
from types import SimpleNamespace

data = '{"openai":{"status":"success","generated_text":"Example of generated_text" }}'

# Parse JSON into an object with attributes corresponding to dict keys.
x = json.loads(data, object_hook=lambda d: SimpleNamespace(**d))
print(x.openai.generated_text)

Example of generated_text


Als we nu alles samenbrengen kunnen we onze `get_answer` van vorige keer aanpassen om of OpenAI, of EdenAI of GPT4All te gebruiken al naargelang.

In [4]:
from gpt4all import GPT4All
model = GPT4All("ggml-model-gpt4all-falcon-q4_0.bin")

Found model file at  /Users/pieter/.cache/gpt4all/ggml-model-gpt4all-falcon-q4_0.bin


objc[32874]: Class GGMLMetalClass is implemented in both /Users/pieter/anaconda3/envs/trendsinai/lib/python3.11/site-packages/gpt4all/llmodel_DO_NOT_MODIFY/build/libreplit-mainline-metal.dylib (0x14a714228) and /Users/pieter/anaconda3/envs/trendsinai/lib/python3.11/site-packages/gpt4all/llmodel_DO_NOT_MODIFY/build/libllamamodel-mainline-metal.dylib (0x14a5dc228). One of the two will be used. Which one is undefined.


falcon_model_load: loading model from '/Users/pieter/.cache/gpt4all/ggml-model-gpt4all-falcon-q4_0.bin' - please wait ...
falcon_model_load: n_vocab   = 65024
falcon_model_load: n_embd    = 4544
falcon_model_load: n_head    = 71
falcon_model_load: n_head_kv = 1
falcon_model_load: n_layer   = 32
falcon_model_load: ftype     = 2
falcon_model_load: qntvr     = 0
falcon_model_load: ggml ctx size = 3872.64 MB
falcon_model_load: memory_size =    32.00 MB, n_mem = 65536
falcon_model_load: ........................ done
falcon_model_load: model size =  3872.59 MB / num tensors = 196


In [5]:
from enum import Enum
import json
from types import SimpleNamespace

class API(Enum):
    OPEN_AI = 1
    GPT4All = 2
    EDEN_AI = 3


def get_answer(prompt, which_model=API.OPEN_AI):
    if which_model == API.GPT4All:
        res = model.generate(prompt)
        return res
    elif which_model == API.OPEN_AI:
        message = [{"role": "user", "content": prompt}]
        res = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=message
        )
        return res.choices[0].message["content"]
    elif which_model == API.EDEN_AI:
        url = "https://api.edenai.run/v2/text/chat"

        payload = {
            "response_as_dict": True,
            "attributes_as_list": False,
            "show_original_response": False,
            "temperature": 0,
            "max_tokens": 1000,
            "providers": "openai",
            "text": f"{prompt}"
        }
        headers = {
            "accept": "application/json",
            "content-type": "application/json",
            "authorization": f"Bearer {edenai_api_key}"
        }

        response = requests.post(url, json=payload, headers=headers)
        responseObject = json.loads(response.text, object_hook=lambda d: SimpleNamespace(**d))
        return responseObject.openai.generated_text;

            

In [6]:
print("GPT4All")
print(get_answer("What are the advantages of GPT4All", API.GPT4All))
print("Open AI")
print(get_answer("What are the advantages of OpenAI", API.OPEN_AI))
print("Eden AI")
print(get_answer("What are the advantages of EdenAI", API.EDEN_AI))

GPT4All
?
GPT4All is a platform that allows users to interact with artificial intelligence (AI) models like GPT-4. Here are some advantages of using GPT4All:

1. Access to AI models: GPT4All provides access to a wide range of AI models, including GPT-4, which is one of the most advanced language models in existence. This gives users the opportunity to interact with cutting-edge AI technology and gain valuable insights into how these models work.
2. Easy integration: GPT4All is easy to use and integrates seamlessly with other applications. Users can access the platform from any device, whether it's a laptop, tablet, or smartphone. This makes it convenient for users who want to interact with AI models on-the-go.
3. Customization options: GPT4All offers a range of customization options that allow users to tailor their interactions with AI models to meet their specific needs. For example,
Open AI
OpenAI offers several advantages that make it a prominent player in the field of artificial in

## LangChain, waarom?

LangChain is een framework dat dient om applicaties die gebruik maken van language models te ontwikkelen. Het laat toe om vrij eenvoudig verschillende componenten aan elkaar te 'chainen' (vandaar de naam), en zo meer gestandaardiseerd allerlei verschillende modellen te gebruiken.

Het is zeer snel, zeer groot geworden (al meer dan 1700 contributors op [github](https://github.com/langchain-ai/langchain) voor de python versie, en nog eens 400 voor de js versie)
Het leuke is dat je snel veel soorten AI modellen kan gebruiken, of makkelijk uitbreiden met vectorstores en andere extra functionaliteit; en je bij een wissel naar een ander model niet per se alles opnieuw dient te programmeren of aan te passen.

Een (voorlopig?) nadeel is dat het zo snel evolueert en groeit dat de documentatie best wel te wensen over laat.  

## Models

We gaan eerst een llm model creëeren, je hebt de keuze tussen een EdenAI of OpenAI model.

In [7]:
from enum import Enum

from langchain.llms import EdenAI, OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.embeddings.edenai import EdenAiEmbeddings
from langchain.embeddings import OpenAIEmbeddings

class API(Enum):
    OPEN_AI = 1
    GPT4All = 2
    EDEN_AI = 3
    

def get_llm(which_model=API.OPEN_AI, temperature = 0.0):
    if which_model == API.OPEN_AI:
        # OpenAI heeft ook een ChatModel, maar dat is niet makkelijk transparent te gebruiken als je ook EdenAI wil gebruiken in dezelfde requests. Als je enkel met OpenAI wenst te werken is dit zeker een goede optie. Je zal dan wel het output formaat moeten aanpassen.
        # return ChatOpenAI(temperature=temperature, model="gpt-3.5-turbo")
        return OpenAI(temperature=temperature)
    elif which_model == API.EDEN_AI:
        return EdenAI(edenai_api_key=edenai_api_key,provider="openai", model="gpt-3.5-turbo-instruct", temperature=temperature, max_tokens=250)

def get_embedding(which_model=API.OPEN_AI):
    if which_model == API.OPEN_AI:
        return OpenAIEmbeddings()
    elif which_model == API.EDEN_AI:
        return EdenAiEmbeddings(edenai_api_key=edenai_api_key,provider="openai")
       

In [8]:
preferred_model = API.EDEN_AI

# llm = get_llm(API.EDEN_AI, 0.0)
# creative_llm = get_llm(API.EDEN_AI, 0.9)
llm = get_llm(preferred_model, 0.0)
creative_llm = get_llm(preferred_model, 0.9)
embeddings = get_embedding(preferred_model)


## Prompts

Om makkelijker prompts te genereren die (deels) variabel zijn heeft LangChain [https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/](PromptTemplates), een PromptTemplate krijgt een string met {variabelen} als `template`, en genereert dan samen met een array van `input_variables` een prompt die kan gebruikt worden voor een llm.



In [9]:
template_string = """Translate the text between three backticks to {language}. \
text: ```{text}```
"""

from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(template_string)

print (prompt_template)

input_variables=['language', 'text'] template='Translate the text between three backticks to {language}. text: ```{text}```\n'


In [10]:
translate_messages = prompt_template.format(
    language="french",
    text="Dit is een tekst in het Nederlands")
print (translate_messages)
llm(translate_messages)

Translate the text between three backticks to french. text: ```Dit is een tekst in het Nederlands```


'\n\n```Ceci est un texte en néerlandais```'

## Output formatting

De standaard output is gewoon een stuk tekst. Vaak wil je echter JSON of een ander gestructureerd formaat om dan makkelijker verder te kunnen werken (en 'chainen')

Als voorbeeld nemen we een review van Disneyland, genomen uit een dataset van [https://www.kaggle.com/](https://www.kaggle.com/)

In [11]:
review = f"""If you've ever been to Disneyland anywhere you'll find Disneyland Hong Kong very similar in the layout when you walk into main street! It has a very familiar feel. One of the rides  its a Small World  is absolutely fabulous and worth doing. The day we visited was fairly hot and relatively busy but the queues moved fairly well."""


review_template = """\
For the following text, extract the following information:

location: Was the review about Disneyland Paris, California, Hong Kong or Unknown

weather: Was there any indication about the weather conditions, answer with \
'TOO HOT', 'HOT', 'RAIN', 'COLD' or 'UNKNOWN' if no information was provided

rides: Extract any sentences about the rides,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
location
weather
rides

text: {text}
"""


In [12]:

from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(review_template)
print(prompt_template)

message = prompt_template.format(text=review)
response = llm(message)
print(response)

type(response)
response.get('location')

input_variables=['text'] template="For the following text, extract the following information:\n\nlocation: Was the review about Disneyland Paris, California, Hong Kong or Unknown\n\nweather: Was there any indication about the weather conditions, answer with 'TOO HOT', 'HOT', 'RAIN', 'COLD' or 'UNKNOWN' if no information was provided\n\nrides: Extract any sentences about the rides,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\nlocation\nweather\nrides\n\ntext: {text}\n"


{
    "location": "Hong Kong",
    "weather": "HOT",
    "rides": ["One of the rides  its a Small World  is absolutely fabulous and worth doing."]
}


AttributeError: 'str' object has no attribute 'get'

Zoals je kan zien krijgen we een `string` terug van het systeem, het zou natuurlijk veel beter zijn als dit een echt JSON object is. We gebruiken hiervoor [StructuredOutputParser](https://python.langchain.com/docs/modules/model_io/output_parsers/structured)

In [13]:

from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

location_schema = ResponseSchema(name="location",
                             description="Was the review about Disneyland \
                                    'Paris', 'California', 'Hong Kong' or 'Unknown'")
weather_schema = ResponseSchema(name="weather",
                                      description="Was there any indication \
                                        about the weather conditions, answer with \
                                        'TOO HOT', 'HOT', 'RAIN', 'COLD' or 'UNKNOWN' \
                                         if no information was provided")
ride_schema = ResponseSchema(name="rides",
                                    description="Extract any sentences about \
                                    the rides, and output them as a comma \
                                    separated Python list.")

response_schemas = [location_schema,
                    weather_schema,
                    ride_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

review_template_2 = """\
For the following text, extract the following information:

location: Was the review about Disneyland Paris, California, Hong Kong or Unknown

weather: Was there any indication about the weather conditions, answer with \
'TOO HOT', 'HOT', 'RAIN', 'COLD' or 'UNKNOWN' if no information was provided

rides: Extract any sentences about the rides,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = PromptTemplate.from_template(template=review_template_2)

messages = prompt.format(text=review, format_instructions=format_instructions)
print(messages)

response = llm(messages)
print(response)

output_dict = output_parser.parse(response)

print(output_dict)
type(output_dict)
output_dict.get('rides')

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"location": string  // Was the review about Disneyland                                     'Paris', 'California', 'Hong Kong' or 'Unknown'
	"weather": string  // Was there any indication                                         about the weather conditions, answer with                                         'TOO HOT', 'HOT', 'RAIN', 'COLD' or 'UNKNOWN'                                          if no information was provided
	"rides": string  // Extract any sentences about                                     the rides, and output them as a comma                                     separated Python list.
}
```
For the following text, extract the following information:

location: Was the review about Disneyland Paris, California, Hong Kong or Unknown

weather: Was there any indication about the weather conditions, answer with 'TOO HOT', 'HOT', 

OutputParserException: Got invalid JSON object. Error: Expecting value: line 1 column 1 (char 0)

## Indexes

### Conversation Memory

Met behulp van een [https://python.langchain.com/docs/modules/memory/types/buffer](Conversation Buffer) krijgt je conversatie een 'geheugen', je hoeft niet langer de prompts in hun geheel mee te geven. (standaard start elke prompt van nul, en kan de AI dus geen rekening houden met wat er reeds eerder gezegd is)

In [14]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.clear()
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True # door verbose aan te zetten zien we de volledige chain en prompts verschijnen
)

In [15]:
print(llm("Hallo, ik ben Pieter"))
print(llm("Hoeveel is 10+2?"))
print(llm("Hoe heet ik?"))



Ik ben een 25-jarige student uit Gent. Ik ben een grote dierenvriend en heb zelf een hond en een kat. Ik ben op zoek naar een bijverdienste en ik zou graag op uw huisdier passen. Ik ben een verantwoordelijke en betrouwbare persoon en ik zal ervoor zorgen dat uw huisdier de beste zorg krijgt. Ik ben beschikbaar om te wandelen, te voeden, te spelen en te verzorgen. Ik ben ook beschikbaar om te overnachten als dat nodig is. Ik kijk ernaar uit om voor uw huisdier te zorgen!

Vanaf € 9 per uur /uur

Leeftijd: 24

Ervaring: 10+ Jaren

Ik ben een 24-jarige studente uit Gent. Ik ben een grote dierenvriend en heb zelf een hond en een kat. Ik ben op zoek naar een bijverdienste en ik zou graag op uw huisdier passen. Ik ben een verantwoordelijke en betrouwbare persoon en ik zal ervoor zorgen dat uw huis


12


Dat kan ik niet weten, ik ben een computerprogramma en heb geen toegang tot persoonlijke informatie.


In [16]:
conversation.predict(input="Hallo, ik ben Pieter")
conversation.predict(input="Hoeveel is 10+2?")
conversation.predict(input="Hoe heet ik?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hallo, ik ben Pieter
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hallo, ik ben Pieter
AI:  Hallo Pieter, ik ben een AI ontwikkeld door OpenAI. Mijn naam is GPT-3 en ik ben geprogrammeerd om te communiceren met mensen zoals jij. Wat kan ik voor je doen?
Human: Hoeveel is 10+2?
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after for

' Je hebt me verteld dat je naam Pieter is, dus ik neem aan dat dat je naam is. Als je een andere naam wilt gebruiken, kun je me dat vertellen en zal ik je zo noemen.'

In [17]:
memory.load_memory_variables({})
print(memory.buffer)

Human: Hallo, ik ben Pieter
AI:  Hallo Pieter, ik ben een AI ontwikkeld door OpenAI. Mijn naam is GPT-3 en ik ben geprogrammeerd om te communiceren met mensen zoals jij. Wat kan ik voor je doen?
Human: Hoeveel is 10+2?
AI:   10+2 is 12. Wist je dat GPT-3 in staat is om complexe wiskundige berekeningen uit te voeren? Het is een van mijn vele vaardigheden.
Human: Hoe heet ik?
AI:  Je hebt me verteld dat je naam Pieter is, dus ik neem aan dat dat je naam is. Als je een andere naam wilt gebruiken, kun je me dat vertellen en zal ik je zo noemen.


Je kan de grootte van je conversation buffer zelf beheren.

In [18]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=False
)

memory.save_context({"input": "Hallo"}, {"output": "Hoe gaat het?"})
memory.save_context({"input": "Alles ok, gewoon beetje saaie les AI"}, {"output": "Hopelijk niet te lang meer dan"})

memory.load_memory_variables({})
print(memory.buffer)

conversation.predict(input="Hallo, ik ben Pieter")

conversation.predict(input="Hoe heet ik?")
print(memory.buffer)

conversation.predict(input="Hoeveel is 10+6?")

conversation.predict(input="Hoe heet ik?")
print(memory.buffer)

Human: Alles ok, gewoon beetje saaie les AI
AI: Hopelijk niet te lang meer dan
Human: Hoe heet ik?
AI:  Je hebt me verteld dat je Pieter heet, dus dat is hoe ik je noem. Maar als je een andere naam wilt gebruiken, kun je me dat vertellen en ik zal je zo noemen.
Human: Hoe heet ik?
AI:  Ik heb geen informatie over jouw naam, dus ik kan niet zeggen hoe je heet. Wil je mij jouw naam vertellen zodat ik je in de toekomst kan herkennen?


In plaats van de buffer te beperken op basis van het aantal vraag-antwoorden, kan je het ook op basis van het aantal tokens doen met behulp van `ConversationTokenBufferMemory`

### oefening 

Gebruik TokenBufferMemory om een deel van een conversatie te beperken, kijk wat er gebeurt als de 'helft' van een zin wegvalt door de limit


Een `ConversationSummaryBufferMemory` gaat niet gewoon alles uit het geheugen gooien als er geen plaats meer is (en alles waarvoor er wel plaats is letterlijk onthouden) maar gaat proberen een samenvatting te maken van wat anders zou verdwijnen.

In [19]:

from langchain.memory import ConversationSummaryBufferMemory

# create a long string
planning = "Na een ontbijt 's ochtends moet je tegen 10u naar \
Gent om op tijd te zijn voor de les MLOps \
Na de lasagne als lunch moet je weer klaar zijn \
voor het interessante Machine Learning Operations \
Maar het hoogtepunt van de dag vormt natuurlijk \
de les Trends in AI waar je het vandaag over LangChain \
zal hebben. \
"

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hoi"}, {"output": "Alles goed?"})
memory.save_context({"input": "Ja hoor, weekend weer gedaan"},
                    {"output": "Cool"})
memory.save_context({"input": "Wat staat er op de planning voor vandaag?"},
                    {"output": f"{planning}"})

memory.load_memory_variables({})

conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

conversation.predict(input="Wat wordt de interessantse les vandaag?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human and AI engage in small talk about their weekend and plans for the day. The AI mentions a busy schedule including a morning class on MLOps, lunch of lasagne, and an interesting lesson on Trends in AI with a focus on LangChain.
Human: Wat wordt de interessantse les vandaag?
AI:

> Finished chain.


' De interessantste les vandaag is Trends in AI met een focus op LangChain. Het is een fascinerende les waarin we zullen leren over de nieuwste ontwikkelingen in AI en hoe LangChain een rol speelt in deze ontwikkelingen. Ik ben erg enthousiast om meer te leren over dit onderwerp. Wat zijn jouw plannen voor vandaag?'

## Chains

Lang*Chain*, er bestaan een heleboel 'Chain' klassen die je toestaan de output van één prompt als input voor de volgende te gebruiken. 

In [20]:
from langchain.chains import LLMChain

prompt = PromptTemplate.from_template("""Wat is de beste naam voor een vak met als onderwerp {onderwerp}?""")

chain = LLMChain(llm=creative_llm, prompt=prompt)
onderwerp = "LangChain, prompt engineering, chatgpt en meer trendy AI dingen"

chain.run(onderwerp)

'\n"AI Innovations: Ontdekkingen in LangChain, Prompt Engineering en ChatGPT" '

Bij een `SimpleSequentialChain` wordt de output van de ene als input voor de volgende gebruikt, zonder dat er nood is aan een key of iets dergelijks. Het systeem gaat er vanuit dat de output van de ene de (enige) input van de andere is.

In [21]:
from langchain.chains import SimpleSequentialChain
from langchain.prompts import PromptTemplate

onderwerp = "LangChain, prompt engineering, chatgpt en meer trendy AI dingen"

# prompt template 1
first_prompt = PromptTemplate.from_template("""Wat is de beste naam voor een vak met als onderwerp {onderwerp}?""")

# Chain 1
chain_one = LLMChain(llm=creative_llm, prompt=first_prompt)

# prompt template 2
second_prompt = PromptTemplate.from_template("""Geef een beschrijving van een 20 tal woorden over het \
    vak:{naam_vak}""")
# chain 2
chain_two = LLMChain(llm=creative_llm, prompt=second_prompt)

overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

overall_simple_chain.run(onderwerp)



> Entering new SimpleSequentialChain chain...


"Next-Gen AI: Innovaties in LangChain, Prompt Engineering en ChatGPT" 
 is een vak dat zich richt op de nieuwste ontwikkelingen op het gebied van kunstmatige intelligentie, met een focus op toegepaste technologieën zoals LangChain, Prompt Engineering en ChatGPT. Studenten leren over de toepassingen en mogelijkheden van deze innovaties en hoe ze deze kunnen gebruiken om complexe problemen op te lossen en menselijke interactie te verbeteren. Het vak biedt een unieke kans om te werken met cutting-edge tools en technieken en om deel uit te maken van de vooruitgang van AI.

> Finished chain.


' is een vak dat zich richt op de nieuwste ontwikkelingen op het gebied van kunstmatige intelligentie, met een focus op toegepaste technologieën zoals LangChain, Prompt Engineering en ChatGPT. Studenten leren over de toepassingen en mogelijkheden van deze innovaties en hoe ze deze kunnen gebruiken om complexe problemen op te lossen en menselijke interactie te verbeteren. Het vak biedt een unieke kans om te werken met cutting-edge tools en technieken en om deel uit te maken van de vooruitgang van AI.'

Als er iets complexere kettingen dienen gevormd te worden, waar een prompt input dient te krijgen van twee verschillende andere prompts bijvoorbeeld, gebruiken we een `SequentialChain`, hier kan je telkens een key associëren met een output, en zo meer controle krijgen over welke output bij welke input gebruikt wordt. 

In [22]:

from langchain.chains import SequentialChain

# dit was een uitvoer van één van de vorige prompts, "Avantgarde Artificial Intelligence Technologies" klinkt wel wat fancier dan "Trends in AI" eigenlijk
vak = """Het vak Avantgarde Artificial Intelligence Technologies is gericht op het ontwikkelen en toepassen van technologie op het gebied van kunstmatig \
intelligentie (AI). Het biedt een grondige kennis over technieken zoals machine learning (ML), deep learning (DL) en natural language processing (NLP) en hun \
toepassingen. Het omvat ook onderwerpen zoals computer vision, robottechnologie, rekenkracht en architectuur voor AI. Studenten leren concepten te verwerken, te \
implementeren en te verbeteren in complexe systemen. Ze krijgen bovendien ook de kans om bestaande AI-oplossingen te evalueren om verbeteringen door te voeren.
"""

# prompt template 1: translate to english
first_prompt = PromptTemplate.from_template(
    "Vertaal de beschrijving van het volgende vak naar het Engels: {vak_beschrijving}"
)
chain_one = LLMChain(llm=creative_llm, prompt=first_prompt,
                     output_key="vak_beschrijving_engels")

second_prompt = PromptTemplate.from_template(
    "Geef een samenvatting van 10 woorden van de volgende beschrijving: {vak_beschrijving_engels}"
)
chain_two = LLMChain(llm=creative_llm, prompt=second_prompt,
                     output_key="samenvatting")


third_prompt = PromptTemplate.from_template(
    "In welke taal is de beschrijving van het volgende vak: {vak_beschrijving}"
)
chain_three = LLMChain(llm=creative_llm, prompt=third_prompt, output_key="taal" )

fourth_prompt = PromptTemplate.from_template(
    "Geef wat extra uitleg over het vak dat hier samengevat is in de opgegeven taal "
    "\n\nSamenvatting: {samenvatting}\nTaal: {taal}"
)
chain_four = LLMChain(llm=creative_llm, prompt=fourth_prompt,
                      output_key="extra_uitleg" )


overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["vak_beschrijving"],
    output_variables=["vak_beschrijving_engels", "samenvatting","extra_uitleg"],
    verbose=True
)

overall_chain(vak)



> Entering new SequentialChain chain...

> Finished chain.


{'vak_beschrijving': 'Het vak Avantgarde Artificial Intelligence Technologies is gericht op het ontwikkelen en toepassen van technologie op het gebied van kunstmatig intelligentie (AI). Het biedt een grondige kennis over technieken zoals machine learning (ML), deep learning (DL) en natural language processing (NLP) en hun toepassingen. Het omvat ook onderwerpen zoals computer vision, robottechnologie, rekenkracht en architectuur voor AI. Studenten leren concepten te verwerken, te implementeren en te verbeteren in complexe systemen. Ze krijgen bovendien ook de kans om bestaande AI-oplossingen te evalueren om verbeteringen door te voeren.\n',
 'vak_beschrijving_engels': '\n\nThe course Avantgarde Artificial Intelligence Technologies focuses on the development and application of technology in the field of artificial intelligence (AI). It provides a thorough knowledge of techniques such as machine learning (ML), deep learning (DL), and natural language processing (NLP) and their applicatio

## RetrievalQA: Q&A over documenten

We hebben al heel wat klassen van langchain de revue zien passeren en kleinere chains gebouwd. Tijd om eens een wat 'echtere' toepassingen na te bootsen.
Nu gaan we een grote(re) hoeveelheid data inladen gebruik makend van een vectorstore; en dan een Q&A systeem over deze data creëeren.

Als voorbeeld nemen we een dataset van Steam games, bekijk eerst de data even in een terminal (of iets dergelijks). 
 

In [23]:
from langchain.document_loaders import CSVLoader

file = 'data/steamgames_10.csv'

loader = CSVLoader(file_path=file, csv_args={
    "delimiter": ",",
    "quotechar": '"',
    "fieldnames":  ["Title","Original Price","Discounted Price","Release Date","Link","Game Description","Recent Reviews Summary","All Reviews Summary","Recent Reviews Number","All Reviews Number","Developer","Publisher","Supported Languages","Popular Tags","Game Features","Minimum Requirements"]
})

data = loader.load()
print(data[1])

page_content="Title: Baldur's Gate 3\nOriginal Price: $29.99\nDiscounted Price: $29.99\nRelease Date: 3 Aug, 2023\nLink: https://store.steampowered.com/app/1086940/Baldurs_Gate_3/?snr=1_7_7_230_150_1\nGame Description: Baldur’s Gate 3 is a story-rich, party-based RPG set in the universe of Dungeons & Dragons, where your choices shape a tale of fellowship and betrayal, survival and sacrifice, and the lure of absolute power.\nRecent Reviews Summary: Overwhelmingly Positive\nAll Reviews Summary: Very Positive\nRecent Reviews Number: - 96% of the 128,900 user reviews in the last 30 days are positive.\nAll Reviews Number: - 94% of the 188,617 user reviews for this game are positive.\nDeveloper: Larian Studios\nPublisher: Larian Studios\nSupported Languages: ['English', 'French', 'German', 'Spanish - Spain', 'Polish', 'Russian', 'Simplified Chinese', 'Turkish', 'Portuguese - Brazil', 'Italian', 'Spanish - Latin America', 'Traditional Chinese', 'Ukrainian']\nPopular Tags: ['RPG', 'Choices Mat

`Embeddings` zorgen ervoor dat we een vector representatie van een stuk tekst kunnen genereren., 

In [24]:
embed = embeddings.embed_query("We volgen de les Trends in AI")
print(len(embed))
print(embed[:5])

1536
[-0.0038772349, -0.03442644, 0.0058666533, -0.007105533, 0.005666728]


We kunnen nu similarity searches uitvoeren gebruik makend van vectorstores. Een makkelijk te hanteren versie is de `DocArrayInMemorySearch`. Zoals de naam suggereert gebeurt alles in memory, dus enkel geschikt als de data niet té groot is, maar heeft het grote voordeel dat we geen externe vector store API's moeten aanspreken of configureren.

In [25]:
from langchain.vectorstores import DocArrayInMemorySearch

db = DocArrayInMemorySearch.from_documents(
    data,
    embeddings
)

query = "give me a strategy game that released in 2023 and has a high rating"

result = db.similarity_search(query)

print(result[0])


page_content="Title: NARAKA: BLADEPOINT\nOriginal Price: Free\nDiscounted Price: Free\nRelease Date: 11 Aug, 2021\nLink: https://store.steampowered.com/app/1203220/NARAKA_BLADEPOINT/?snr=1_7_7_230_150_1\nGame Description: Dive into the legends of the Far East in NARAKA: BLADEPOINT; team up with friends in fast-paced melee fights for a Battle Royale experience unlike any other. Find your playstyle with a varied cast of heroes with unique skills. More than 20 million players have already joined the fray, play free now!\nRecent Reviews Summary: Mostly Positive\nAll Reviews Summary: \nRecent Reviews Number: - 75% of the 146,529 user reviews for this game are positive.This product has experienced one or more periods of off-topic review activity.  Based on your preferences, the reviews within these periods have been excluded from this product's Review Score.\nAll Reviews Number: \nDeveloper: 24 Entertainment\nPublisher: NetEase Games Global\nSupported Languages: ['English', 'Simplified Chine

Zo wordt er wel een gelijkaardig resultaat gevonden, op basis van de vector embedding, maar om wat 'menselijkere' output te krijgen, in de vorm van vraag-antwoord, gebruiken we de `RetrievalQA`

Die zal een llm combineren met onze vectordatabank.

In [26]:
from langchain.chains import RetrievalQA

retriever = db.as_retriever() # we gebruiken de vectorstore als een Retriever, andere interface om basically hetzelfde te bereiken

qa_stuff = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

response = qa_stuff.run(query)
print(response)




> Entering new RetrievalQA chain...

> Finished chain.
 Baldur's Gate 3, released on August 3, 2023, has a very positive rating on Steam. It is a story-rich, party-based RPG set in the universe of Dungeons & Dragons, with popular tags such as "Choices Matter" and "Strategy."


Er bestaat ook een makkelijkere manier om hetzelfde te bekomen, een `VectorStoreIndexCreator`, die veel korter juist hetzelfde doet.
Alles nog eens opnieuw (dus ook document loaden e.d., om te tonen hoe weinig code nodig is)

In [27]:

from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

file = 'data/steamgames_10.csv'
# loader = CSVLoader(file_path=file)
loader = CSVLoader(file_path=file, csv_args={
    "delimiter": ",",
    "quotechar": '"',
    "fieldnames":  ["Title","Original Price","Discounted Price","Release Date","Link","Game Description","Recent Reviews Summary","All Reviews Summary","Recent Reviews Number","All Reviews Number","Developer","Publisher","Supported Languages","Popular Tags","Game Features","Minimum Requirements"]
})
from langchain.indexes import VectorstoreIndexCreator

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])


In [28]:
query = "give me three fairly new games with a high rating that are discounted formatted as a markdown list"
response = index.query(query)

display(Markdown(response))


- [Cyberpunk 2077](https://store.steampowered.com/app/1091500/Cyberpunk_2077/?snr=1_7_7_230_150_1) (Original Price: $29.99, Discounted Price: $14.99, Release Date: 9 Dec, 2020, Reviews Summary: Very Positive)
- [Baldur's Gate 3](https://store.steampowered.com/app/1086940/Baldurs_Gate_3/?snr=1_7_7_230_150_1) (Original Price: $29.99, Discounted Price: $29.99, Release Date: 3 Aug, 2023, Reviews Summary: Very Positive)
- [Call of Duty®](https://store.steampowered.com/app/1938090/Call_of_Duty/?snr=1_7_7_230_150_1) (Original Price: Free, Discounted Price: Free, Release Date: 27 Oct, 2022, Reviews Summary: Mixed)

In [29]:
query = ("give me three games form the 2020's which cost less than $15 formatted as a markdown list")
response = index.query(query)

display(Markdown(response))



- Cyberpunk 2077 ($14.99)
- Call of Duty® (Free)
- Rust ($19.99)

## Evalueren

Hoe kunnen we nu zien hoe goed deze chains hun werk doen? We zouden manueel kunnen door onze data gaan en goede voorbeelden vinden, daar vragen op loslaten, en kijken of de antwoorden voldoen.
Maar dat schaalt natuurlijk absoluut niet. Dus we gaan een llm chain gebruiken om onze llm chain te evalueren.



In [30]:
from langchain.evaluation.qa import QAGenerateChain

example_gen_chain = QAGenerateChain.from_llm(llm)

# genereert een warning, maar de oprichter van langchain zelf zegt dat we ze mogen negeren :)
# (zal dus in de toekomst wel nog aangepast worden)
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

print(new_examples[1])
print(data[1])

/Users/pieter/anaconda3/envs/trendsinai/lib/python3.11/site-packages/langchain/chains/llm.py:308: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'qa_pairs': {'query': "What is the release date for Baldur's Gate 3?", 'answer': "The release date for Baldur's Gate 3 is 3 Aug, 2023."}}
page_content="Title: Baldur's Gate 3\nOriginal Price: $29.99\nDiscounted Price: $29.99\nRelease Date: 3 Aug, 2023\nLink: https://store.steampowered.com/app/1086940/Baldurs_Gate_3/?snr=1_7_7_230_150_1\nGame Description: Baldur’s Gate 3 is a story-rich, party-based RPG set in the universe of Dungeons & Dragons, where your choices shape a tale of fellowship and betrayal, survival and sacrifice, and the lure of absolute power.\nRecent Reviews Summary: Overwhelmingly Positive\nAll Reviews Summary: Very Positive\nRecent Reviews Number: - 96% of the 128,900 user reviews in the last 30 days are positive.\nAll Reviews Number: - 94% of the 188,617 user reviews for this game are positive.\nDeveloper: Larian Studios\nPublisher: Larian Studios\nSupported Languages: ['English', 'French', 'German', 'Spanish - Spain', 'Polish', 'Russian', 'Simplified Chinese', 'Tur

Dan kunnen we deze vragen uitvoeren en zien of de antwoorden correct gegeven worden, maar manueel schaalt dat natuurlijk weer niet zo goed.

In [31]:
examples = list(map(lambda i: i["qa_pairs"], new_examples))
qa_stuff.run(examples[1]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


' 3 Aug, 2023'

In [32]:
import langchain

langchain.debug = True
qa_stuff.run(examples[1]["query"])
langchain.debug = False

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What is the release date for Baldur's Gate 3?"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "What is the release date for Baldur's Gate 3?",
  "context": "Title: Baldur's Gate 3\nOriginal Price: $29.99\nDiscounted Price: $29.99\nRelease Date: 3 Aug, 2023\nLink: https://store.steampowered.com/app/1086940/Baldurs_Gate_3/?snr=1_7_7_230_150_1\nGame Description: Baldur’s Gate 3 is a story-rich, party-based RPG set in the universe of Dungeons & Dragons, where your choices shape a tale of fellowship and betrayal, survival and sacrifice, and the lure of absolute power.\nRecent Reviews Summary: Overwhelmingly Positive\nAll Reviews Summary: Very Positive\nRecent Reviews Number: - 96% of the 128,900 user reviews in the last 3

Dus we gaan al deze vragen door QAChain sturen 

In [33]:

predictions = qa_stuff.apply(examples)



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


En dan kunnen we ze allemaal tegelijk laten evalueren met behulp van een QAEvalChain

In [34]:

from langchain.evaluation.qa import QAEvalChain

eval_chain = QAEvalChain.from_llm(llm)

graded_outputs = eval_chain.evaluate(examples, predictions)

for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['results'])
    print()

graded_outputs[1]

Example 0:
Question: What is the minimum system requirement for this game?
Real Answer: The minimum system requirement for this game can be found under the "Minimum Requirements" section in the document.
Predicted Answer:  Requires a 64-bit processor and operating system | OS: |  64-bit Windows 7 | Processor: |  AMD FX 4350 or Equivalent, Intel Core i3 6300 or Equivalent | Memory: |  6 GB RAM | Graphics: |  AMD Radeon™ HD 7730, NVIDIA GeForce® GT 640 | DirectX: |  Version 11 | Network: |  Broadband Internet connection | Storage: |  56 GB available space | Additional Notes: |  ~3.8GB for 1 localized language
Predicted Grade:  CORRECT

Example 1:
Question: What is the release date for Baldur's Gate 3?
Real Answer: The release date for Baldur's Gate 3 is 3 Aug, 2023.
Predicted Answer:  3 Aug, 2023
Predicted Grade:  CORRECT

Example 2:
Question: What is the release date for Counter-Strike: Global Offensive?
Real Answer: The release date for Counter-Strike: Global Offensive is August 21, 20

{'results': ' CORRECT'}

## Agents

Agents zijn een vrij nieuwe toevoeging aan LangChain, en laten toe om andere soorten externe bronnen toe te voegen. Bijvoorbeeld een rekenmodule, wikipedia of een zoekmachine.

In [35]:

from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI


tools = load_tools(["llm-math","wikipedia"], llm=llm)

agent= initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

agent("Wat is 15% van 250")
agent("Wat is de cosinus van 45º")



> Entering new AgentExecutor chain...
Could not parse LLM output: 

Thought: I should use the calculator tool to calculate the percentage.
Action:
{
  "action": "Calculator",
  "action_input": "15% * 250"
}


Observation: Invalid or incomplete response
Thought:Could not parse LLM output:  I should try again with a different input format.
Action:
{
  "action": "Calculator",
  "action_input": "15% of 250"
}


Observation: Invalid or incomplete response
Thought:Could not parse LLM output:  I should try using the calculator tool with a different input format.
Action:
{
  "action": "Calculator",
  "action_input": "15% of 250"
}



Observation: Invalid or incomplete response
Thought:Could not parse LLM output: Could not parse LLM output:  I should try using the calculator tool with a different input format.
Action:
{
  "action": "Calculator",
  "action_input": "15% of 250"
}



Observation: Invalid or incomplete response
Thought:Could not parse LLM output: Could not parse LLM output: Could

{'input': 'Wat is de cosinus van 45º',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [36]:
question = "Welk boek eindigt met de quote 'The Party told you to reject the evidence of your eyes and ears. It was their final, most essential command.' "
result = agent(question) 



> Entering new AgentExecutor chain...
Could not parse LLM output: 

Thought: I should use Wikipedia to search for the book that ends with this quote.
Action:
{
  "action": "Wikipedia",
  "action_input": "The Party told you to reject the evidence of your eyes and ears. It was their final, most essential command."
}


Observation: Invalid or incomplete response
Thought:Could not parse LLM output:  I should try searching for the quote in a different format.
Action:
{
  "action": "Wikipedia",
  "action_input": "The Party told you to reject the evidence of your eyes and ears."
}


Observation: Invalid or incomplete response
Thought:Could not parse LLM output:  I should try searching for the quote in a different format.
Action:
{
  "action": "Wikipedia",
  "action_input": "1984 (roman)"
}



Observation: Invalid or incomplete response
Thought:Could not parse LLM output: Could not parse LLM output:  I should try searching for the quote in a different format.
Action:
{
  "action": "Wikipedia